In [ ]:
%matplotlib inline
import numpy as np
import cv2
import matplotlib.pyplot as plt

def write_ply(fn, verts, colors): 
    ply_header = '''ply
    format ascii 1.0
    element vertex %(vert_num)d
    property float x
    property float y
    property float z
    property uchar red
    property uchar green
    property uchar blue
    end_header
    '''
    verts = verts.reshape(-1, 3)
    colors = colors.reshape(-1, 3)
    verts = np.hstack([verts, colors])
    with open(fn, 'wb') as f:
        f.write((ply_header % dict(vert_num=len(verts))).encode('utf-8'))
        np.savetxt(f, verts, fmt='%f %f %f %d %d %d ')
        
img_dir = './imgs'

---
#### P6: Stereo images


---
<div class="alert alert-info">
<p>
University of Applied Sciences Munich<br>
Dept of Electrical Enineering and Information Technology<br>
Institute for Applications of Machine Learning and Intelligent Systems (IAMLIS)<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;(c) Alfred Schöttl 2024<br>
    
</p>
</div>

In this notebook, we analyze rectified stereo images and compute the depth of certain image areas based on disparities. Parts of the tutorial are based on the official OpenCV documentation, see samples, see https://github.com/opencv/opencv/blob/master/samples/python/stereo_match.py .


### 1. Acquire rectified images
We have already prepared __*rectified images*__ for you. These are obtained by two cameras in a parallel stereo view configuration. Alternatively, you could use two cameras in a general configuration, measure their relative pose by a __*stereo calibration*__ procedure and warp the images so that the epipolar lines are horizontal lines.

Plot the two images side by side and compare them.

_Hint_: Use `axs[0].imshow` and do not switch off the axes for a better comparison.

In [ ]:
imgL = cv2.imread(os.path.join(img_dir, 'left-aloe-image-1.png'))
imgR = cv2.imread(os.path.join(img_dir, 'right-aloe-image-2.png'))

...

### 2. Find disparities
There are various ways to find corresponding image points. One is to rely on eminent imagefeatures points based on methods like SIFT, SURF, HOG. Another possibility, especially for rectified images, is __*block matching*__. We simply scan on the epipolar line for the corresponding, best matching image point. Read the documentation of `StereoSGBM_create` and `compute` to understand how to perform block matching.

_Hint:_ To display the disparities, scale the `disp` image values between 0 and 1. The minimum disparity is 0, the maximum is `num_disp`.

In [ ]:
# look into the documentation and try different parameters to improve the block matching result
window_size = 3
min_disp = 0
num_disp = 80-min_disp
stereo = cv2.StereoSGBM_create(minDisparity = min_disp, 
    numDisparities = num_disp,
    blockSize = 16,
    P1 = 8*3*window_size**2,
    P2 = 32*3*window_size**2,
    disp12MaxDiff = 1,
    uniquenessRatio = 10,
    speckleWindowSize = 100,
    speckleRange = 32
)

disp = stereo.compute(imgL, imgR)

...

### 3. Generate a point cloud
The function `reprojectImageTo3D(d,Q)` takes the image of disparities and computes the depths of all pixels based on a matrix $Q$ (called _disparity to 3D matrix_) which needs to be known or estimated. 

Let $\tilde p=(x,y,d,1)$ be the homogeneous vector consisting of the image coordinates $x,y$ of a feature and its disparity $d$. To make the formulas independent of the baseline length, let the disparity be given in units of the baseline length, $d/b$. Check that our formulas for the parallel-view reconstruction can essentially be written as $Q \tilde p$ with the 4x4 matrix $Q$ as described below.  

A good guess for the focal length $f$ of our cameras is 250 (pixels). 

_Hint_: Pixels whose depths could not determined are assigned the depth `np.inf` (infinity) or -1. We will remove these pixels from the point cloud.

In [ ]:
h, w = imgL.shape[:2]
f = 250
Q = np.array([[1, 0, 0, -0.5*w],
              [0,-1, 0,  0.5*h], # turn points 180 deg around x-axis,
              [0, 0, 0,     -f], # so that the y-axis looks up
              [0, 0, 1,      0]])

points = cv2.reprojectImageTo3D(disp, Q)

colors = cv2.cvtColor(imgL, cv2.COLOR_BGR2RGB)
mask = disp > -1
out_points, out_colors = points[mask], colors[mask]
# just for safety: remove all infinity entries
I = ~np.isinf(out_points).any(axis=1)
out_points, out_colors = out_points[I], out_colors[I]

### 4. Render the result
We store the point cloud in a widely used file format (ply). Try a point cloud viewer such as meshlab, https://www.meshlab.net/ (free and available for all OSs), just load the .ply file. Additionally, you can try to render the point cloud with the matplotlib 3D viewer.

In [ ]:
write_ply('out.ply', out_points, out_colors)     # you may use this file later for visualization (see below)

%matplotlib notebook
plt.axes(projection='3d')
N = out_points.shape[0]
I = np.random.randint(0, high=N-1, size=int(N*0.05))   # subsample to accelerate the rendering, increase the rate
x, y, z = out_points[I,0], out_points[I,1], out_points[I,2],
plt.gca().scatter(x, y, z, c=z, marker=".", cmap="viridis");
# you will obtain a better feeling for the result using the ply file!